In [1]:
import pandas as pd
import xarray as xr
import numpy as np

In [ ]:
# อ่านข้อมูล radiation
ssrd_ds = xr.open_dataset(f'../data/CAMS/cams_forecast/cams_forecast_ssrd_2023.grib',engine='cfgrib')

# อ่านข้อมูล metadata
site_df = pd.read_csv('../data/DeDe_49stations/DeDe_49sites_metadata.csv')

In [ ]:
# เตรียม datetime index สำหรับใช้เลือกข้อมูล forecast จากแต่ละวัน
st = '2023-01-01'
en = '2023-12-31'
time = pd.date_range(st, en, freq='1D')

for i, row in site_df.iterrows():
    print(f'Processing site {row["Site_id"]}')
    forecast_df = pd.DataFrame()

    for date in time:
        # เลือกดึงข้อมูลจากไฟล์ grib โดยใช้ข้อมูล latitude longtitude จากไฟล์ metadata
        df = ssrd_ds.sel(time=f'{str(date.date())}',latitude=row['lat'],longitude=row['lng'],method='nearest').to_dataframe()

        # ปรับข้อมูลค่ารังสีจากแบบ ค่ารังสีสะสม ให้เป็น ค่ารังสี ณ ช่วงเวลานั้น
        df['ssrd'] = (df['ssrd'] - df['ssrd'].shift(1).fillna(0)) / 3600
        df['ssrdc'] = (df['ssrdc'] - df['ssrdc'].shift(1).fillna(0)) / 3600
        forecast_df = pd.concat([forecast_df, df], axis=0)

    # ลบ column และ index ที่ไม่จำเป็นออก
    forecast_df.reset_index(drop=True,inplace=True)
    forecast_df.drop(columns=['time','number','surface','latitude', 'longitude'], inplace=True)

    # เปลี่ยนชื่อ column และ ปรับ timezone ข้อมูล
    forecast_df.rename(columns={'valid_time': 'Datetime'}, inplace=True)
    forecast_df.set_index('Datetime', inplace=True)
    forecast_df = forecast_df.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    forecast_df.reset_index(inplace=True)

    # save ข้อมูลในรูปแบบ csv
    forecast_df.to_csv(f'../data/CAMS/cams_forecast/all_site_1hr/cams_forecast_1hr_2023_site{row['Site_id']:02d}.csv', index=False)

In [ ]:
# เตรียม datetime index สำหรับใช้เลือกข้อมูล forecast จากแต่ละวัน
st = '2023-01-01'
en = '2023-12-31'
time = pd.date_range(st, en, freq='1D')

for i, row in site_df.iterrows():
    print(f'Processing site {row["Site_id"]}...')
    forecast_df = pd.DataFrame()
    for date in time:
        # เลือกดึงข้อมูลจากไฟล์ grib โดยใช้ข้อมูล latitude longtitude จากไฟล์ metadata
        df = ssrd_ds.sel(time=f'{str(date.date())}',latitude=row['lat'],longitude=row['lng'],method='nearest').to_dataframe()

        # ปรับข้อมูลค่ารังสีจากแบบ ค่ารังสีสะสม ให้เป็น ค่ารังสี ณ ช่วงเวลานั้น
        df['ssrd'] = (df['ssrd'] - df['ssrd'].shift(1).fillna(0)) / 3600
        df['ssrdc'] = (df['ssrdc'] - df['ssrdc'].shift(1).fillna(0)) / 3600

        # upsampling จากรายชั่วโมงให้เป็นราย 15 นาที
        df.set_index('valid_time', inplace=True)
        df = df.resample('15min').asfreq()
        df['ssrd'] = df['ssrd'].interpolate(method='linear')
        df['ssrdc'] = df['ssrdc'].interpolate(method='linear')
        df.reset_index(inplace=True)

        forecast_df = pd.concat([forecast_df, df], axis=0)

    # ลบ column และ index ที่ไม่จำเป็นออก
    forecast_df.reset_index(drop=True,inplace=True)
    forecast_df.drop(columns=['time','number','surface','latitude', 'longitude'], inplace=True)
    forecast_df.rename(columns={'valid_time': 'Datetime'}, inplace=True)

    # เปลี่ยนชื่อ column และ ปรับ timezone ข้อมูล
    forecast_df.set_index('Datetime', inplace=True)
    forecast_df = forecast_df.tz_localize('UTC').tz_convert('Asia/Bangkok').tz_localize(None)
    forecast_df.reset_index(inplace=True)

    # save ข้อมูลในรูปแบบ csv
    forecast_df.to_csv(f'../data/CAMS/cams_forecast/all_site_15min/cams_forecast_15min_2023_site{row['Site_id']:02d}.csv', index=False)